In [1]:
import os, sys
from PIL import Image
import numpy as np
import imageio
import json
import random
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm, trange
import dotmap

import matplotlib.pyplot as plt

import run_nerf
import importlib 
importlib.reload(run_nerf)

from load_blender import load_blender_data


In [20]:
!export CUDA_VISIBLE_DEVICES=7
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = 'cpu'


np.random.seed(0)
DEBUG = False

In [16]:
parser = run_nerf.config_parser()
args = parser.parse_args("--config ./configs/cuboid.txt")
args

Namespace(N_importance=128, N_rand=1024, N_samples=64, basedir='./logs', chunk=32768, config='./configs/cuboid.txt', datadir='../datasets/volumetric/results_500', dataset_type='blender', expname='blender_one_cuboid', factor=8, ft_path=None, half_res=True, i_embed=0, i_img=100, i_print=100, i_testset=50000, i_video=50000, i_weights=10000, lindisp=False, llffhold=8, lrate=0.0005, lrate_decay=500, multires=10, multires_views=4, netchunk=65536, netdepth=8, netdepth_fine=8, netwidth=256, netwidth_fine=256, no_batching=True, no_ndc=False, no_reload=False, perturb=1.0, precrop_frac=0.5, precrop_iters=500, raw_noise_std=0.0, render_factor=0, render_only=False, render_test=False, shape='greek', spherify=False, testskip=8, use_viewdirs=True, white_bkgd=True)

In [21]:
# args.ckpt_path = "./logs/cuboid_exp_2/050000.tar"
args.datadir = "../datasets/volumetric/results_500"
args.expname = 'cuboid_exp_2'
args.render_only = True
args.render_factor = 0
args.device = device
# args.config = './configs/cuboid.txt'

render_kwargs_train, render_kwargs_test, start, grad_vars, optimizer = run_nerf.create_nerf(args)

Found ckpts ['./logs/cuboid_exp_2/010000.tar', './logs/cuboid_exp_2/020000.tar', './logs/cuboid_exp_2/030000.tar', './logs/cuboid_exp_2/040000.tar', './logs/cuboid_exp_2/050000.tar', './logs/cuboid_exp_2/060000.tar', './logs/cuboid_exp_2/070000.tar', './logs/cuboid_exp_2/080000.tar', './logs/cuboid_exp_2/090000.tar', './logs/cuboid_exp_2/100000.tar', './logs/cuboid_exp_2/110000.tar', './logs/cuboid_exp_2/120000.tar', './logs/cuboid_exp_2/130000.tar', './logs/cuboid_exp_2/140000.tar', './logs/cuboid_exp_2/150000.tar', './logs/cuboid_exp_2/160000.tar', './logs/cuboid_exp_2/170000.tar', './logs/cuboid_exp_2/180000.tar', './logs/cuboid_exp_2/190000.tar', './logs/cuboid_exp_2/200000.tar']
Reloading from ./logs/cuboid_exp_2/200000.tar
Not ndc!


In [5]:
!ls ./logs/cuboid_exp_2/

010000.tar  190000.tar
020000.tar  200000.tar
030000.tar  args.txt
040000.tar  config.txt
050000.tar  cuboid_exp_2_spiral_050000_disp.mp4
060000.tar  cuboid_exp_2_spiral_050000_rgb.mp4
070000.tar  cuboid_exp_2_spiral_100000_disp.mp4
080000.tar  cuboid_exp_2_spiral_100000_rgb.mp4
090000.tar  cuboid_exp_2_spiral_150000_disp.mp4
100000.tar  cuboid_exp_2_spiral_150000_rgb.mp4
110000.tar  cuboid_exp_2_spiral_200000_disp.mp4
120000.tar  cuboid_exp_2_spiral_200000_rgb.mp4
130000.tar  imgs
140000.tar  testset_050000
150000.tar  testset_100000
160000.tar  testset_150000
170000.tar  testset_200000
180000.tar


In [6]:
images, poses, render_poses, hwf, i_split = load_blender_data(args.datadir, args.half_res, args.testskip)
print('Loaded blender', images.shape, render_poses.shape, hwf, args.datadir)
i_train, i_val, i_test = i_split

if args.white_bkgd:
    images = images[...,:3]*images[...,-1:] + (1.-images[...,-1:])
else:
    images = images[...,:3]

Loaded blender (514, 400, 400, 4) torch.Size([40, 4, 4]) [400, 400, 979.1950876133984] ../datasets/volumetric/results_500


In [7]:
near = 1.
far = 200.

In [22]:
# Cast intrinsics to right types
H, W, focal = hwf
H, W = int(H), int(W)
hwf = [H, W, focal]
K = np.array([
    [focal, 0, 0.5*W],
    [0, focal, 0.5*H],
    [0, 0, 1]
])
# del run_nerf
importlib.reload(run_nerf)


i_train, i_val, i_test = i_split
render_poses = np.array(poses[i_test])
render_poses = torch.Tensor(render_poses).to(device)

args.render_test = True

if args.render_only:
    print('RENDER ONLY')
    with torch.no_grad():
        if args.render_test:
            # render_test switches to test poses
            images_render = images[i_test]#.to(args.device)
        else:
            # images_render is smoother render_poses path
            images_render = None

        testsavedir = os.path.join(args.datadir, args.expname, 
                                   'renderonly_{}_{:06d}'.format('test' if args.render_test else 'path', 
                                                                 start))
        os.makedirs(testsavedir, exist_ok=True)
        print('test poses shape', render_poses.shape)

        rgbs, disps = run_nerf.render_path(render_poses, hwf, K, args.chunk, render_kwargs_test, 
                                  gt_imgs=images_render, 
                                  savedir=testsavedir, render_factor=args.render_factor)
        print('Done rendering', testsavedir)
#         imageio.mimwrite(os.path.join(testsavedir, 'video.mp4'), to8b(rgbs), fps=30, quality=8)

#         return

  0%|          | 0/7 [00:00<?, ?it/s]

RENDER ONLY
test poses shape torch.Size([7, 4, 4])
0 0.0018699169158935547
cpu cpu


  0%|          | 0/7 [08:13<?, ?it/s]


ValueError: too many values to unpack (expected 4)